In [41]:
profit = [3, 5, 4]
constraintProfit = [[[2, 3, 0],8],[[3, 2, 2], 15],[[0, 4,5], 10]]

In [42]:
def simplex(obj, constraints):
    # Make copies of the input to avoid modifying the original data
    obj = obj.copy()
    constraints = [constraint.copy() for constraint in constraints]

    # Add slack variables
    num_constraints = len(constraints)
    for i in range(num_constraints):
        # Ensure constraints[i][0] has enough zeros to accommodate slack variables
        constraints[i][0].extend([0] * (len(obj) + num_constraints - len(constraints[i][0])))
        constraints[i][0][len(obj) + i] = 1
        obj.append(0)

    # Initialize Cb with the coefficients of slack variables (all zeros initially)
    Cb = [0] * num_constraints

    # Initialize zj and cj-zj
    zj = [0] * len(obj)
    cj_zj = [obj[i] - zj[i] for i in range(len(obj))]

    while max(cj_zj) > 0:  # Continue until there are no positive cj-zj values
        # Find the pivot column (entering variable)
        pivotColumn = cj_zj.index(max(cj_zj))

        # Find the pivot row (leaving variable)
        minRatio = float('inf')
        pivotRow = -1
        for i in range(num_constraints):
            if constraints[i][0][pivotColumn] > 0:  # Avoid division by zero or negative values
                ratio = constraints[i][1] / constraints[i][0][pivotColumn]
                if ratio < minRatio:
                    minRatio = ratio
                    pivotRow = i


        # Update Cb with the coefficient of the entering variable
        Cb[pivotRow] = obj[pivotColumn]

        # Perform pivoting
        pivot = constraints[pivotRow][0][pivotColumn]
        constraints[pivotRow][0] = [x / pivot for x in constraints[pivotRow][0]]
        constraints[pivotRow][1] /= pivot

        for i in range(num_constraints):
            if i != pivotRow:
                factor = constraints[i][0][pivotColumn]
                constraints[i][0] = [constraints[i][0][j-1] - factor * constraints[pivotRow][0][j-1] for j in range(len(constraints[i][0]))]
                constraints[i][1] -= factor * constraints[pivotRow][1]

        # Update zj and cj-zj
        zj = [sum(Cb[j] * constraints[j][0][i] for j in range(num_constraints)) for i in range(len(obj))]
        cj_zj = [obj[i] - zj[i] for i in range(len(obj))]

        # Print the current tableau
        print("Current Tableau:")
        for constraint in constraints:
            print(constraint)
        print(f'zj: {zj}')
        print(f'cj-zj: {cj_zj}')
        print(f"{'-' * 10}")


In [43]:
simplex(profit, constraintProfit)

Current Tableau:
[[-0.75, 2.0, 0.0, -3.75, 1.0, 0.0], 0.5]
[[-0.5, 3.0, 0.0, -0.5, 0.0, 0.0, 1.0], 10.0]
[[0.0, 1.0, 1.25, 0.0, 0.0, 0.0, 0.0, 0.25], 2.5]
zj: [0.0, 5.0, 6.25, 0.0, 0.0, 0.0]
cj-zj: [3.0, 0.0, -2.25, 0.0, 0.0, 0.0]
----------


ZeroDivisionError: float division by zero